# 🧹 Cleaning Data, Part 2: Cleaning in Pandas

*Same idea, different style.*

Let's take that same list of amount descriptions, but this time make it a Pandas `Series`:

In [1]:
import pandas as pd

In [2]:
amounts = pd.Series([
    "   1,000.31   doLLaRs   ",
    "54 cents  ",
    "33 CENTS",
    "$10"
])

amounts

0       1,000.31   doLLaRs   
1                  54 cents  
2                    33 CENTS
3                         $10
dtype: object

In Pandas, you can use the same methods we explored in plain Python, but you access them through each Series' `.str` attribute:

- __Plain__: `my_string.strip()`
- __Pandas__: `my_series.str.strip()`

🧹🧹🧹

- __Plain__: `my_string.replace("a", "b")`
- __Pandas__: `my_series.str.replace("a", "b")`

... and so on

Let's try it out:

In [3]:
amounts

0       1,000.31   doLLaRs   
1                  54 cents  
2                    33 CENTS
3                         $10
dtype: object

... versus:

In [4]:
amounts.str.strip("$ ")

0    1,000.31   doLLaRs
1              54 cents
2              33 CENTS
3                    10
dtype: object

In [5]:
amounts.str.replace(",", "")

0       1000.31   doLLaRs   
1                 54 cents  
2                   33 CENTS
3                        $10
dtype: object

In [6]:
amounts.str.lower()

0       1,000.31   dollars   
1                  54 cents  
2                    33 cents
3                         $10
dtype: object

In [7]:
amounts.str.split(" ")

0    [, , , 1,000.31, , , doLLaRs, , , ]
1                        [54, cents, , ]
2                            [33, CENTS]
3                                  [$10]
dtype: object

`.str.get(num)` is equivalent to equivalent to `my_list[num]` in plain Python, and can be helpful to combine with `.str.split(...)`:

In [8]:
amounts.str.split(" ")

0    [, , , 1,000.31, , , doLLaRs, , , ]
1                        [54, cents, , ]
2                            [33, CENTS]
3                                  [$10]
dtype: object

In [9]:
amounts.str.split(" ").str.get(0)

0       
1     54
2     33
3    $10
dtype: object

In [10]:
amounts.str.split(" ").str.get(-1)

0         
1         
2    CENTS
3      $10
dtype: object

Now, let's put it all together to get the __quantities__ from the amount descriptions (ignoring, for now, whether they represent the number of dollars or cents):

In [11]:
(
    amounts
    .str.strip("$ ")
    .str.replace(",", "")
    .str.split()
    .str.get(0)
    .astype(float)    
)

0    1000.31
1      54.00
2      33.00
3      10.00
dtype: float64

We're going to want to reuse this approach, so let's __put it in a function__:

In [12]:
def get_quantity(amounts):
    return (
        amounts
        .str.strip("$ ")
        .str.replace(",", "")
        .str.split()
        .str.get(0)
        .astype(float)
    )

Now we can call that function on our pandas `Series`:

In [13]:
get_quantity(amounts)

0    1000.31
1      54.00
2      33.00
3      10.00
dtype: float64

Or, equivalently:

In [14]:
amounts.pipe(get_quantity)

0    1000.31
1      54.00
2      33.00
3      10.00
dtype: float64

## What about handling dollars vs. cents?

We have a couple of options for implementing this:

- A very Pandas-y way (works, but a bit overly complex)
- Just writing a normal Python function, and passing it to `amounts.apply(...)`

In [15]:
def get_conversion(amt):
    if "$" in amt or "dollar" in amt.lower():
        conversion = 1
    elif "cent" in amt.lower():
        conversion = 0.01
    else:
        raise ValueError(f"Cannot determine unit for {amt}")
    return conversion

In [16]:
amounts.apply(get_conversion)

0    1.00
1    0.01
2    0.01
3    1.00
dtype: float64

Let's tie it all together, creating a `DataFrame` with:

- The raw, original description
- The quantity extracted
- The conversion factor

... which we'll use to convert to the dollars-normalized values, so we can `sum` it all up.

In [17]:
amounts_df = pd.DataFrame({
    "raw": amounts,
    "quantity": get_quantity(amounts),
    "conversion": amounts.apply(get_conversion),
})

amounts_df

,raw,quantity,conversion
0,"1,000.31 doLLaRs",1000.31,1.00
1,54 cents,54.00,0.01
2,33 CENTS,33.00,0.01
3,$10,10.00,1.00


In [18]:
amounts_df["dollars"] = amounts_df["quantity"] * amounts_df["conversion"]

amounts_df

,raw,quantity,conversion,dollars
0,"1,000.31 doLLaRs",1000.31,1.00,1000.31
1,54 cents,54.00,0.01,0.54
2,33 CENTS,33.00,0.01,0.33
3,$10,10.00,1.00,10.00


In [19]:
amounts_df["dollars"].sum()

np.float64(1011.18)

In [20]:
amounts_df["dollars"].sum().item()

1011.18

---

---

---